# Example Sample Hotel and Flight Booker Agent 

Ang solusyon na ito ay tutulong sa iyo na mag-book ng mga tiket ng eroplano at hotel. Ang senaryo ay isang biyahe mula London Heathrow LHR noong Pebrero 20, 2024 papuntang New York JFK na babalik noong Pebrero 27, 2025, nakasakay sa economy class gamit ang British Airways lamang. Nais ko ng pananatili sa isang Hilton hotel sa New York, mangyaring ibigay ang mga gastos para sa flight at hotel.


# I-initialize ang Azure AI Agent Service at kunin ang impormasyon ng configuration mula sa **.env**

### **.env**

Gumawa ng isang .env na file

**.env** ay naglalaman ng connection string ng Azure AI Agent Service, ang modelong ginagamit ng AOAI, at ang katugmang Google API Search service API, ENDPOINT, atbp.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Pangalan ng Iyong Azure AI Agent Service Model Deployment"

[**NOTE**] Kailangan mo ng modelong may 100,000 Rate Limit (Tokens kada minuto) at Rate Limit na 600 (Request kada minuto)

  Makukuha mo ang model sa Microsoft Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Pangalan ng Iyong Azure AI Agent Service Project Connection String"

  Makukuha mo ang project connection string sa overview ng iyong proyekto sa AI Foundry Portal Screen.

- **SERPAPI_SEARCH_API_KEY** = "Iyong SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Iyong SERPAPI Search Endpoint"

Para makuha ang Model Deployment Name at Project Connection String ng Azure AI Agent Service, kailangan mong gumawa ng Azure AI Agent Service. Inirerekomenda na gamitin ang [template na ito](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) upang likhain ito nang direkta （***Tandaan:*** Ang Azure AI Agent Service ay kasalukuyang nakatakda lamang sa mga limitadong rehiyon. Inirerekomenda na tumukoy sa [link na ito](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) upang itakda ang rehiyon)

Kinakailangan ng Agent na ma-access ang SERPAPI. Inirerekomenda na magparehistro gamit ang [link na ito](https://serpapi.com/searches). Pagkatapos ng pagpaparehistro, makakakuha ka ng natatanging API KEY at ENDPOINT


# Setup 

Para patakbuhin ang notebook na ito, kailangan mong tiyakin na na-install mo na ang mga kinakailangang library sa pamamagitan ng pagpapatakbo ng `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Ang iyong bersyon ng Semantic Kernel ay dapat hindi bababa sa 1.27.2.


I-load ang iyong .env file na setting at mga resources, mangyaring tiyakin na naidagdag mo ang iyong mga susi at setting at nakagawa ng lokal na .env file.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Mag-log in sa Azure

Kailangan mo nang mag-log in sa Azure. Buksan ang terminal at patakbuhin ang sumusunod na utos:

```bash
az login
```

Ang utos na ito ay hihilingin sa iyo na ilagay ang iyong mga kredensyal sa Azure, na nagpapagana sa serbisyo ng Azure AI Agent nang tama.


# Paliwanag:
Ito ay isang variable na nag-iimbak ng API key para sa pag-access ng SERP (Search Engine Results Page) API service. Ang API key ay isang natatanging identifier na ginagamit upang i-authenticate ang mga kahilingan na kaugnay ng iyong account.

Layunin: Ang layunin ng linyang ito ay i-imbak ang API key sa isang variable upang magamit ito sa pag-authenticate ng mga kahilingan sa SERP API service. Kinakailangan ang API key upang ma-access ang serbisyo at magsagawa ng mga paghahanap.
Paano Kumuha ng SERP API Key: Upang makakuha ng SERP API key, sundin ang mga pangkalahatang hakbang na ito sa https://serpapi.com (ang eksaktong mga hakbang ay maaaring mag-iba depende sa partikular na SERP API service na iyong ginagamit):

Pumili ng SERP API Service: Mayroong ilang SERP API services na available, tulad ng SerpAPI, Google Custom Search JSON API, at iba pa. Piliin ang pinakanaaangkop sa iyong mga pangangailangan.

Mag-sign Up para sa Isang Account: Pumunta sa website ng napiling SERP API service at mag-sign up para sa isang account. Maaaring kailanganin mong magbigay ng ilang pangunahing impormasyon at i-verify ang iyong email address.

Gumawa ng API Key: Pagkatapos mag-sign up, mag-log in sa iyong account at pumunta sa seksyon ng API o dashboard. Hanapin ang opsyon upang gumawa o mag-generate ng bagong API key.
Kopyahin ang API Key sa iyong .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: Ito ay isang variable na nag-iimbak ng base URL para sa SERP API endpoint. Ang pangalan ng variable na BASE_URL ay isang kaugalian na ginagamit upang ipahiwatig na ang URL na ito ang panimulang punto para sa paggawa ng mga API request.
'https://serpapi.com/search':

Ito ang aktwal na string ng URL na nakatalaga sa variable na BASE_URL. Ito ay kumakatawan sa endpoint para sa pagsasagawa ng mga search query gamit ang SERP API.

# Purpose:
Ang layunin ng linyang ito ay upang magtakda ng isang constant na naglalaman ng base URL para sa SERP API. Ang URL na ito ay gagamitin bilang panimulang punto para sa pagbuo ng mga API request upang magsagawa ng mga operasyon sa paghahanap.

# Usage:
Sa pamamagitan ng pagtatakda ng base URL sa isang variable, madali mo itong magagamit muli sa buong iyong code tuwing kailangan mong gumawa ng request sa SERP API. Ginagawa nitong mas madaling mapanatili ang iyong code at binabawasan ang panganib ng mga pagkakamali mula sa pag-hardcode ng URL sa maraming lugar. Ang kasalukuyang halimbawa ay https://serpapi.com/search?engine=bing na gumagamit ng Bing search API. Maaaring pumili ng ibang API sa https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Paliwanag:

Dito matatagpuan ang iyong plugin code.

Pagdeklara ng Klase: `class BookingPlugin`: Nagdeklara ng isang klase na pinangalanang BookingPlugin na naglalaman ng mga metodo para sa pag-book ng mga hotel at flight.

Paraan sa Pag-book ng Hotel:

- `@kernel_function(description="booking hotel")`: Isang decorator na naglalarawan sa function bilang kernel function para sa pag-book ng mga hotel.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Nagdeklara ng isang metodo para sa pag-book ng mga hotel na may mga annotated na parameter at uri ng return.

Ang metodo ay bumubuo ng diksyunaryo ng mga parameter para sa kahilingan sa pag-book ng hotel at nagpapadala ng GET request sa SERP API. Sine-check nito ang status ng tugon at ibinabalik ang mga hotel properties kung matagumpay, o None kung nabigo ang kahilingan.

Paraan sa Pag-book ng Flight:

- `@kernel_function(description="booking flight")`: Isang decorator na naglalarawan sa function bilang kernel function para sa pag-book ng mga flight.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Nagdeklara ng isang metodo para sa pag-book ng mga flight na may mga annotated na parameter at uri ng return.

Ang metodo ay bumubuo ng mga diksyunaryo ng mga parameter para sa outbound at return flight requests at nagpapadala ng GET requests sa SERP API. Sine-check nito ang status ng tugon at idinadagdag ang impormasyon ng flight sa resulta kung matagumpay, o nagpi-print ng error message kung nabigo ang kahilingan. Ibinabalik ng metodo ang resulta na naglalaman ng impormasyon ng flight.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Paliwanag:
Mga Import na Pahayag: Mag-import ng mga kinakailangang module para sa mga kredensyal ng Azure, AI agent, nilalaman ng chat message, papel ng awtor, at dekorador ng kernel function.

Asynchronous na Tagapamahala ng Konteksto: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Ito ay nagse-set up ng asynchronous na tagapamahala ng konteksto upang hawakan ang mga kredensyal ng Azure at lumikha ng isang AI agent client.

Pangalan ng Agent at mga Tagubilin: 
- `AGENT_NAME = "BookingAgent"`: Dine-define ang pangalan ng agent.
- `AGENT_INSTRUCTIONS = """..."""`: Nagbibigay ng detalyadong mga tagubilin para sa agent kung paano hawakan ang mga booking request.

Gumawa ng Agent Definition: `agent_definition = await client.agents.create_agent(...)`: Lumilikha ng isang agent definition gamit ang tinukoy na modelo, pangalan, at mga tagubilin.

Gumawa ng AzureAI Agent: `agent = AzureAIAgent(...)`: Lumilikha ng AzureAI agent gamit ang client, agent definition, at ang tinukoy na plugin.

Gumawa ng Thread: `thread: AzureAIAgentThread | None = None`: Gumawa ng thread para sa agent. Hindi kinakailangan na lumikha muna ng thread - kung ang halaga na `None` ang ibibigay, isang bagong thread ang malilikha sa unang pagtawag at ibabalik bilang bahagi ng tugon.

Mga Input ng User: `user_inputs = ["..."]`: Dine-define ang listahan ng mga user input para iproseso ng agent.

Sa finally block, tanggalin ang thread at agent upang linisin ang mga resources.


# Pagpapatunay ng pagkakakilanlan

Ang klase na `DefaultAzureCredential` ay bahagi ng Azure SDK para sa Python. Nagbibigay ito ng isang default na paraan upang magpatunay ng pagkakakilanlan sa mga serbisyo ng Azure. Sinusubukan nitong magpatunay gamit ang iba't ibang mga pamamaraan sa isang partikular na pagkakasunod-sunod, tulad ng mga environment variable, managed identity, at mga kredensyal ng Azure CLI.

Asynchronous Operations: Ang aio na module ay nagpapahiwatig na sinusuportahan ng klase na DefaultAzureCredential ang mga asynchronous na operasyon. Ibig sabihin, maaari mo itong gamitin kasama ang asyncio upang magsagawa ng mga kahilingan sa pagpapatunay ng hindi nakaharang na paraan.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Pagtatanggi**:
Ang dokumentong ito ay isinalin gamit ang AI na serbisyo sa pagsasalin [Co-op Translator](https://github.com/Azure/co-op-translator). Bagaman nagsusumikap kami para sa katumpakan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o di-tumpak na impormasyon. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na pangunahing sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
